# LSTM- PowerConsumption

## Project Description
The objective is to build a Long Short-Term Memory (LSTM) recurrent neural network to predict the current global active power at the time step (t), given prior measurements at the time step (t-1).

## Attributes
1. date: Date in format dd/mm/yyyy
2. time: Time in format hh:mm:ss
3. global_active_power: household global minute-averaged active power (in kilowatt)
4. global_reactive_power: household global minute-averaged reactive power (in kilowatt)
5. voltage: minute-averaged voltage (in volt)
6. global_intensity: household global minute-averaged current intensity (in ampere)
7. sub_metering_1: energy sub-metering No. 1 (in watt-hour of active energy)
8. sub_metering_2: energy sub-metering No. 2 (in watt-hour of active energy)
9. sub_metering_3: energy sub-metering No. 3 (in watt-hour of active energy)

## Imports

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense

2023-11-19 13:51:59.305630: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Prepare the dataset

In [2]:
data = pd.read_csv("household_power_consumption.txt", sep=';', parse_dates={'datetime': ['Date', 'Time']}, infer_datetime_format=True, index_col='datetime')

/var/folders/0n/3jh9862d73j9wkdg1y54jxf80000gn/T/ipykernel_3084/3735213747.py:1: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("household_power_consumption.txt", sep=';', parse_dates={'datetime': ['Date', 'Time']}, infer_datetime_format=True, index_col='datetime')


Drop missing values

In [3]:
data = data.dropna()

Extract the target variable (global_active_power)

In [4]:
target = data['Global_active_power']

Select features

In [5]:
features = data[['Global_reactive_power', 'Voltage', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']]

## Data Normalization

In [6]:
scaler_target = MinMaxScaler()
scaler_features = MinMaxScaler()

target_scaled = scaler_target.fit_transform(target.values.reshape(-1, 1))
features_scaled = scaler_features.fit_transform(features)

## Define function to create sequences for LSTM

In [7]:
def create_sequences(data, target, time_steps):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:(i + time_steps)])
        y.append(target[i + time_steps])
    return np.array(X), np.array(y)

## Create Sequences

In [8]:
time_steps = 10
X, y = create_sequences(features_scaled, target_scaled, time_steps)

## Split the data into training and testing sets

In [9]:
split = int(0.8 * len(X))
X_train, X_test, y_train, y_test = X[:split], X[split:], y[:split], y[split:]

## Build the LSTM model

In [10]:
model = Sequential()
model.add(LSTM(units=50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

## Train the model

In [11]:
model.fit(X_train, y_train, epochs=2, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/2
51232/51232 [==============================] - 400s 8ms/step - loss: 5.5930e-04 - val_loss: 3.5034e-04
Epoch 2/2
51232/51232 [==============================] - 320s 6ms/step - loss: 4.9746e-04 - val_loss: 3.3463e-04


## Make predictions on the test set

In [12]:
predictions = model.predict(X_test)
predictions_inv = scaler_target.inverse_transform(predictions)
y_test_inv = scaler_target.inverse_transform(y_test.reshape(-1, 1))
rmse = np.sqrt(np.mean(np.square(predictions_inv - y_test_inv)))
print("Root Mean Squared Error (RMSE):", rmse)

12808/12808 [==============================] - 34s 3ms/step
Root Mean Squared Error (RMSE): 0.20206362537234845
